In [ ]:
""" CURRENTLY NOT WORKING AS I RAN OUT OF FREE OPEN SOURCE MODELS THAT ARE CAPABLE FOR THIS"""

""" 
Automated Financial Data Extraction script 

Description:
This script reads financial data from a local Excel file, sends it to an
LLM via the Groq API for structured data extraction, and saves the
resulting metrics into a clean CSV file.

"""

# 1. Library Imports 
import os 
import re 
import io
import time 
import random 
import pandas as pd 
import matplotlib.pyplot as plt 
from groq import Groq, APIError, AuthenticationError



In [ ]:
""" 
API Configuration (Set your API key)

"""
API_KEY = os.getenv("GROQ_API_KEY", "gsk_xxxxxxxxx") # Paste key here if not using env vars
API_PROVIDER = "Groq"
LLM_MODEL = "whisper-large-v3" # Using Llama 3.3- 70B on Groq


In [239]:
""" 
Set the full path to your input Excel file and desired output csv file 
Examples:
    -windows: "C:\\Users\\YourUser\\Documents\\financial_data.xlsx"

"""

EXCEL_FILE_PATH = r"C:\Users\Hruthikesh\Downloads\Tata Motors.xlsx"
OUTPUT_CSV_PATH = "Financial_Data.csv"



In [240]:
""" 1. Helper Function """

def read_excel_to_text(file_path: str) -> str | None:
    """
    Reads all sheets from an Excel file and concatenates them into a single
    text string for the LLM prompt.

    Args:
        file_path (str): The path to the input Excel file.

    Returns:
        str | None: A string containing the content of all sheets, or None if
                    an error occurs.
    """
    print(f"📂 Reading Excel file: {os.path.basename(file_path)}")
    try:
        # Use openpyxl engine, which is robust for .xlsx files
        xls = pd.ExcelFile(file_path, engine='openpyxl')
        all_sheets_content = []
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            # Add a clear separator for the LLM to understand sheet boundaries
            sheet_header = f"--- SHEET: {sheet_name} ---\n\n"
            sheet_content = df.to_string(index=False)
            all_sheets_content.append(sheet_header + sheet_content)

        full_content = "\n\n".join(all_sheets_content)
        print(f"✅ Excel file read successfully ({len(full_content):,} characters)")
        return full_content
    except FileNotFoundError:
        print(f"❌ ERROR: The file was not found at '{file_path}'. Please check the path.")
        return None
    except Exception as e:
        print(f"❌ ERROR: An unexpected error occurred while reading the Excel file: {e}")
        return None


In [241]:
""" 2. Helper Function """
# The main response logic 

def extract_financial_data_with_llm(excel_content: str, api_key: str) -> tuple[str | None, dict | None]:
    """
    Sends the Excel data to the LLM API with a specific prompt and returns
    the extracted CSV content. Includes retry logic for transient errors.
    """
    print(f"🤖 Sending data to {API_PROVIDER} ({LLM_MODEL}) for extraction...")

    # --- PROMPT ENHANCEMENT ---
    # We now use a "system" prompt to set the AI's role and a "user" prompt
    # for the specific task. This is a more robust way to control the output.
    system_prompt = """
    You are a highly specialized data extraction and research bot.
    Your function is to receive text data, find supplemental information from your own knowledge, and convert everything into a single CSV format.
    You MUST NOT provide any explanation, commentary, or code.
    Your entire response MUST be the raw CSV data and nothing else.
    """

    # --- RESTRUCTURED USER PROMPT ---
    user_prompt = f"""
    🔹 Input Excel File Structure
    The provided text contains financial statement data (P&L, Balance Sheet, etc.). Metrics are in rows, years in columns.

    🔹 Output CSV Requirements
    Create one single CSV file.
    Columns must be: year,date,equity_share_capital,reserves,profit_before_tax,stock_price
    Rows must represent years 2016–2025 only.
    The date must always be March 15 in YYYY-03-15 format.
    If any value is missing, leave it blank (NA).

    🔹 Data Cleaning Rules
    - Normalise all financial statement numbers into a consistent unit (₹ Crore).
    - The 'stock_price' should be the price per share in ₹ (Indian Rupees), not in Crores.
    - Remove commas, dashes, or symbols from all numbers.
    - Keep values as floats (not strings).

    🔹 Column Naming Rules
    - Equity Share Capital → equity_share_capital
    - Reserves → reserves
    - Profit Before Tax → profit_before_tax
    - Stock Price → stock_price

    🎯 Task
    1.  **Extract Data from Excel**: From the provided Excel content, extract the 'equity_share_capital', 'reserves', and 'profit_before_tax'.
    2.  **Find External Data**: For the 'stock_price' column, use your knowledge to find the approximate closing stock price for **Reliance Industries** on the National Stock Exchange (NSE) on or around **March 15th** for each year from 2016 to 2025. This data is NOT in the Excel file; you must find it. For future years (like 2025), leave the price blank.
    3.  **Combine and Format**: Combine all data and output ONLY the CSV content. Start directly with the CSV header.

    Here is the Excel file content:

    {excel_content}
    """

    retries = 3
    delay = 5  # seconds

    for attempt in range(retries):
        try:
            client = Groq(api_key=api_key)
            chat_completion = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                model=LLM_MODEL,
                temperature=0.0,
                max_tokens=2000,
                top_p=1,
            )
            response_content = chat_completion.choices[0].message.content
            usage_data = chat_completion.usage
            print("✅ Data extracted successfully from LLM.")
            return response_content, usage_data.dict()

        except AuthenticationError:
            print("❌ ERROR: Authentication failed. Please check your API key.")
            return None, None
        except APIError as e:
            print(f"🟠 WARNING: API Error occurred (Attempt {attempt + 1}/{retries}): {e}")
            if attempt < retries - 1:
                print(f"   Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("❌ ERROR: All API retries failed.")
                return None, None
        except Exception as e:
            print(f"❌ ERROR: An unexpected error occurred during the API call: {e}")
            return None, None

    return None, None
    

In [242]:
""" 3. Helper Function """

def save_and_validate_csv(csv_content: str, output_path: str) -> bool:
    """
    Cleans the LLM response, saves it as a CSV file, and performs a basic
    validation on the output.

    Args:
        csv_content (str): The raw CSV string from the LLM.
        output_path (str): The path to save the output CSV file.

    Returns:
        bool: True if saving and validation were successful, False otherwise.
    """
    print(f"💾 Saving CSV to: {output_path}")
    try:
        # --- Clean the LLM response ---
        # The LLM might wrap the CSV in markdown code blocks (```csv ... ```).
        # This regex robustly extracts the content inside.
        match = re.search(r"```(?:csv\n)?(.*?)```", csv_content, re.DOTALL)
        if match:
            cleaned_csv = match.group(1).strip()
        else:
            # If no markdown block is found, assume the output is clean.
            cleaned_csv = csv_content.strip()

        if not cleaned_csv:
            print("❌ ERROR: LLM returned an empty response after cleaning.")
            return False

        # --- Save to CSV and display preview ---
        # Use io.StringIO to read the string directly into a pandas DataFrame
        df = pd.read_csv(io.StringIO(cleaned_csv))

        # --- Basic Validation ---
        expected_columns = ['year', 'date', 'equity_share_capital', 'reserves', 'profit_before_tax']
        if not all(col in df.columns for col in expected_columns):
            print(f"🟠 WARNING: CSV columns do not match expected format.")
            print(f"   Expected: {expected_columns}")
            print(f"   Got: {list(df.columns)}")
            # Proceed with saving anyway, but flag it to the user.
        else:
            print("✅ CSV structure validated successfully.")

        df.to_csv(output_path, index=False)
        print("✅ CSV file saved successfully.")
        print("\n📄 Preview of extracted data:")
        print(df.head())
        return True

    except pd.errors.ParserError:
        print("❌ ERROR: Failed to parse the LLM output as CSV. The format might be incorrect.")
        print("   Raw LLM Output:")
        print("   " + "-"*20)
        print(csv_content)
        print("   " + "-"*20)
        return False
    except Exception as e:
        print(f"❌ ERROR: An unexpected error occurred while saving the CSV: {e}")
        return False


In [243]:
# Reading and changing the file
def update_data():
    global Financial_Data
    Financial_Data = pd.read_csv("Financial_Data.csv")
    Financial_Data.set_index('year')
    Financial_Data['net_worth'] = Financial_Data['equity_share_capital'] + Financial_Data['reserves']
    Financial_Data.to_csv('Financial_Data.csv', index=False)




In [244]:
# Calculate Average increase in profit_before_tax year-on-year 
def avg_pbt_inc_per(Financial_Data, column_name = 'profit_before_tax'):
    global avg_pbt_per 

    column = Financial_Data[column_name]
    avg_pbt_per = 0
    for i in range(len(column) - 1):
        pbt_inc_per = ((column.iloc[i + 1] - column.iloc[i]) / column.iloc[i + 1]) * 100
        avg_pbt_per += pbt_inc_per/ 9 

    print("The average increase in 'profit_before_tax' is", f"{avg_pbt_per:.2f}%", "year-on-year for past 10 years")
    return float(round(avg_pbt_per, 2))


In [245]:
# Choose a discount value 
def choose_discount_value():
    while True:
        
        try:
            global discount_value
            get_discount_value = input("Choose a discount value between (2 to 20), choose two values for approximation: ")
            discount_value = list(map(int, get_discount_value.replace(",", " ").split()))

            if len(discount_value) != 2:
                print("Enter two discount values, as per risk adjustments")
                continue

            discount_value.sort()

            if discount_value[0] < 2 or discount_value[1] > 20:
                print("Enter value based on the risk judgement")
                continue

            if discount_value[1] - discount_value[0] > 7:
                print("The chosen range may not give correct value")
                continue 

        except ValueError:
            print("Enter valid discount value")
            continue 

        break 
        
    print(f"You have chosen discount value is from {min(discount_value)} to {max(discount_value)}")
    

In [246]:
# Enter the current price 
def enter_current_price():
    while True:
        global current_price
        try:
            current_price = float(input("Enter current price of the stock: "))
            if current_price <= 0:
                print("Enter valid price: ")
                continue 
        except ValueError:
            print("Enter valid stock price.")
        break 
    print(f"The current stock price is Rs.{current_price}")
    return round(current_price, 2)

    

In [247]:
# Predict the price growth over the period of ten years 


def cal_price_growth():
    current_networth = int(Financial_Data.loc[Financial_Data['year'] == 2025, 'net_worth'].values[0])
    current_earnings = int(Financial_Data.loc[Financial_Data['year'] == 2025, 'profit_before_tax'].values[0])
    current_equity_capital = int(Financial_Data.loc[Financial_Data['year'] == 2025, 'equity_share_capital'].values[0])
    
    global future_price
    future_price = [current_price]
    year = 1 
    price_next_year = current_price
    while year < 10: 
        price_next_year += (( current_earnings * (1 + (avg_pbt_per - random.randint(min(discount_value), max(discount_value))))) + (current_networth)) / current_equity_capital
        
        future_price.append(round(price_next_year, 2))
        year += 1
    
    print("The stock price after 10 years will be", round(price_next_year, 2))




In [248]:
# Plot the graph 

def plot_price_graph():
    global prices
    prices = list(Financial_Data['stock_price']) +  future_price
    x = 2016
    years = [x + i for i in range(20)]
    constat_value = current_price

    plt.figure(figsize=(10, 6))
    plt.plot(years, prices, marker= 'o', linestyle='-', label='Prices over Time')
    plt.axhline(y=constat_value, color = 'r', linestyle = '--', label=f'Current Price = {constat_value}')

    plt.xlabel('Years')
    plt.ylabel('Prices')
    plt.title('Price Over Years')
    plt.legend()
    plt.grid(True)
    
    
    plt.show()


In [249]:
# Calculate Margin of Safety 
def margin_of_safety():
    mos = ((int(prices[-1]) - current_price) / int(prices[-1])) * 100 

    if mos > 0:
        print(f"You have a good margin of safety of {mos:.2f}% ")

    if mos <= 0:
        print(f"You are in risk margin of safety of {abs(mos):.2f}% ")

In [250]:
""" Main Function """

def main():
    """
    Main function to orchestrate the Excel to CSV conversion workflow.
    """
    print("=" * 60)
    print(f"📊 EXCEL TO CSV CONVERTER USING {API_PROVIDER.upper()}")
    print("=" * 60)

    #1. Input Validation
    if API_KEY == "gsk_your_api_key_here" or not API_KEY:
        print("❌ CRITICAL: API key is not set. Please update the `API_KEY` variable.")
        return

    if not os.path.exists(EXCEL_FILE_PATH):
        print(f"❌ CRITICAL: Excel file not found at '{EXCEL_FILE_PATH}'.")
        print("   Please update the `EXCEL_FILE_PATH` variable.")
        return

    #2. Read Excel File
    excel_text = read_excel_to_text(EXCEL_FILE_PATH)
    if not excel_text:
        return # Error message already printed in the function

    print("-" * 60)

    #3. Call LLM for Extraction
    csv_from_llm, usage = extract_financial_data_with_llm(excel_text, API_KEY)
    if not csv_from_llm:
        return # Error message already printed in the function

    print("-" * 60)

    #4. Save and Validate CSV
    success = save_and_validate_csv(csv_from_llm, OUTPUT_CSV_PATH)

    print("-" * 60)

    #5. Final Report
    if success:
        if usage:
            print("📈 API Usage & Cost Information:")
            total_tokens = usage.get('total_tokens', 0)
            print(f"   - Prompt Tokens:     {usage.get('prompt_tokens', 0):,}")
            print(f"   - Completion Tokens: {usage.get('completion_tokens', 0):,}")
            print(f"   - Total Tokens Used: {total_tokens:,}")
            # Note: Groq is currently free, but this helps for future-proofing.
            # Example cost calculation for other APIs: cost = (total_tokens / 1_000_000) * price_per_million_tokens
        print("\n" + "=" * 60)
        print("✅ CONVERSION COMPLETED SUCCESSFULLY!")
        print("=" * 60)
    else:
        print("\n" + "=" * 60)
        print("❌ CONVERSION FAILED. Please review the error messages above.")
        print("=" * 60)

    #6. Calculation
    update_data()
    print("-" * 60)
    avg_pbt_inc_per(Financial_Data)
    print("-" * 60)
    choose_discount_value()
    print("-" * 60)
    enter_current_price()
    print("-" * 60)
    cal_price_growth()
    print("-" * 60)
    plot_price_graph()
    print("-" * 60)
    margin_of_safety()
    print("-" * 60)



    


In [251]:
# script excecution 
if __name__ == "__main__":
    main()

📊 EXCEL TO CSV CONVERTER USING GROQ
📂 Reading Excel file: Tata Motors.xlsx
✅ Excel file read successfully (37,098 characters)
------------------------------------------------------------
🤖 Sending data to Groq (whisper-large-v3) for extraction...
🟠 WARNING: API Error occurred (Attempt 1/3): Error code: 400 - {'error': {'message': 'The model `whisper-large-v3` does not support chat completions', 'type': 'invalid_request_error'}}
   Retrying in 5 seconds...
🟠 WARNING: API Error occurred (Attempt 2/3): Error code: 400 - {'error': {'message': 'The model `whisper-large-v3` does not support chat completions', 'type': 'invalid_request_error'}}
   Retrying in 5 seconds...
🟠 WARNING: API Error occurred (Attempt 3/3): Error code: 400 - {'error': {'message': 'The model `whisper-large-v3` does not support chat completions', 'type': 'invalid_request_error'}}
❌ ERROR: All API retries failed.
